In [ ]:
import sisl
import os
import matplotlib.pyplot as plt
import netCDF4 as nc4
%matplotlib inline

# 1. Introduction #

In this tutorial we will showcase some of the features of [`Inelastica`](https://github.com/tfrederiksen/inelastica/), using a simple model to discuss tunneling through a CO molecule adsorbed on Cu.

On real Cu surfaces CO is known to adsorb on the top site in an upright configuration with the C atom towards the surface.

This is the general setup we will work with:

<img src="setup.png">

The 1D Cu electrodes are characterized by the lattice constant *a*. The electrodes are defined as 4-atom blocks of the chains to avoid coupling next-nearest layer coupling between the principal layers. The separation between "substrate" (left electrode) and "STM tip" (right electrode) is defined as *L*.

In the following, we will use [`sisl`](https://github.com/zerothi/sisl/) to construct geometries and [`siesta`](https://gitlab.com/siesta-project/siesta/) to relax geometries and determine the electronic Hamiltonian from DFT with finite differences.

Finally, we will use different scripts from [`Inelastica`](https://github.com/tfrederiksen/inelastica/) to compute core physical properties
* vibrational modes and energies
* eigenchannel scattering states
* inelastic transport characteristics (IETS spectra)

# 2. SIESTA calculations #

## Electrodes ##

To keep things as computationally light as possible, we consider the CO molecule between one-dimensional Cu chains as described above.

Let us begin with looking at the electronic structure of the electrodes.

In [ ]:
a = 2.56 # Cu lattice constant
s = 7.00 # transverse cell separation
Cu = sisl.Atom('Cu', tag='Cu.mpn')

# primitive cell of the electrode
folder = 'PRIM'
sc = sisl.SuperCell([a, s, s], nsc=[3, 1, 1])
elec = sisl.Geometry([0, 0, 0], atoms=Cu, sc=sc)
elec.write(f'{folder}/STRUCT.fdf')
#os.system(f'cd {folder} && siesta RUN.fdf |tee RUN.out')

We can readily look at the electronic bands with `sisl`

In [ ]:
H = sisl.get_sile(f'{folder}/<TSHS-file>').read_hamiltonian()
band = sisl.BandStructure(H, [[0., 0., 0.], [1/2, 1/2, 0]], 101, [r'\Gamma', 'X'])
eigs = band.apply.array.eigh()
lk = band.lineark()
plt.plot(lk, eigs, 'b');
plt.xlabel('Gamma-X'); 
plt.ylabel('$E-E_F$ (eV)');
plt.ylim(-3, 3);

**Questions:**
* What is the symmetry character of the bands?
* What controls their alignment and band width relative to the Fermi energy?
* How does this band structure compare with a 3D Cu crystal?

## Geometry relaxation of device region ##

Next we build the central device region consisting of left electrode, CO molecule, and right electrode. Conventionally we call the geometry relaxation step to determine the device geometry `CGrun` (but this is just a name). Note that we here choose periodic boundary conditions and use a standard `MD.TypeOfRun CG` with `siesta` to relax the CO coordinates (keeping all Cu atoms frozen):

In [ ]:
folder = 'CGrun'
C = sisl.Atom('C', tag='C.mpn')
O = sisl.Atom('O', tag='O.mpn')
# CO molecule
sc = sisl.SuperCell([4, s, s], nsc=[3, 1, 1])
CO = sisl.Geometry([[-0.6503, 0, 0], [0.5853, 0, 0]], atoms=[C, O], sc=sc)
left = elec.tile(6, axis=0)
device = left.append(CO, 0).append(left, 0)
device.write(f'{folder}/STRUCT.fdf')
#os.system(f'cd {folder} && siesta RUN.fdf |tee RUN.out')

**Questions:**
* What is the CO bond length in this setup?
* What are the residual forces on the C, O and surface Cu atoms?

## Finite displacements run ##

To determine the dynamical properties of the CO molecule (normal modes and vibrational energies), as well as the electron-phonon couplings (EPC), we use finite displacements along the cartesian axes for a selected set of **dynamical atoms** within the device. In our case we consider just the motion of CO (atom indices 7 and 8 in the device, `siesta` counting).

`Inelastica` comes with a script (called `setupFCrun`) to prepare a finite-difference run `FCrun` for `siesta` based on the input parameters and geometry from the relaxation `CGrun` above:

In [ ]:
folder = 'FCrun'
#os.system(f'setupFCrun --FCfirst 7 --FClast 8 CGrun {folder}')
#os.system(f'cd {folder} && siesta RUN.fdf |tee RUN.out')

**Questions:**
* Look at the `siesta` input file `RUN.fdf`: The `setupFCrun` script has prepended lines to this file. What do they mean?
* Which `.TSHS` files were generated in the `siesta` run?
* What is contained in the `*.FC` output file?
* The finite displacement runs over atoms is inherently parallel. Try to split the above calculations into two separate runs, say `FCrun_7` for `atom 7` and `FCrun_8` for `atom 8`.  

In order to compute EPCs with an atom-centered, non-orthogonal LCAO basis, one needs to correct for the fact that the overlap matrix changes between the configurations mapped out during the finite-difference `FCrun`. Therefore, if the EPCs are to be computed, `Inelastica` expects to find also an overlap `OSrun` folder providing the necessary info via some separate `siesta` calculations. Again, `Inelastica` comes with a script (called `setupOSrun`) to prepare the input files: 

In [ ]:
folder = 'OSrun'
#os.system(f'setupOSrun CGrun {folder}')
for i in range(1, 7):
    print(f'Doing run {i}/6')
    #os.system(f'cd {folder} && siesta RUN_{i}.fdf |tee RUN_{i}.out')
    

**Questions:**
* The *setupOSrun* has generated six input files (RUN_i.fdf) and structures (STRUCT_i.fdf). What are the differences between these setups?
* Why are these calculations extremely fast compared to a normal SIESTA run?

# 3. Transport calculation (TranSIESTA/TBTRANS) #

In the *CGrun* folder above we relaxed the geometry using periodic boundary conditions. We will now use this exact geometry but extend it for a device region with a **single** molecule between semi-infinite electrodes. In other words, set up the inputs required for a TranSIESTA run (eg. to compute the transmission function with *tbtrans*):

First, we define the electrode block as a 4-atom repetitions of the primitive cell. This is to eliminate interactions between nearest-neighbor cells only:

In [ ]:
# transiesta cell for both electrodes
n = 4 # atoms in electrode block
folder = 'ELEC'
elec.tile(n, axis=0).write(f'{folder}/STRUCT.fdf')
#os.system(f'cd {folder} && siesta RUN.fdf |tee RUN.out')

In [ ]:
folder = 'TSrun'
# read the relaxed structure:
device = sisl.get_sile('<path-to-device.XV>').read_geometry() 
device.write(f'{folder}/STRUCT.fdf')
#os.system(f'cd {folder} && siesta RUN.fdf |tee RUN.out')
#os.system(f'cd {folder} && tbtrans RUN.fdf |tee RUN.TBT.out')

Let us visualize the transmission function from *tbtrans*:

In [ ]:
tbt = sisl.get_sile(f'{folder}/<TBT.nc>')
plt.plot(tbt.E, tbt.transmission());
plt.xlabel('$E - E_F$ (eV)'); plt.ylabel('Transmission T(E)');

**Questions:**
* What is the tunneling probability of electrons near the Fermi level in our setup?
* Can some features in the calculated transmission be related to the electrode band structure?

# 4. Inelastica #

## EigenChannels ##

Before discussing vibrations and inelastic effects, let us first understand the scattering states of our system. These can be computed with the *EigenChannels* script in *Inelastica*:

In [ ]:
folder = 'ECrun'
#os.system(f'EigenChannels -F 1 -L 14 -f TSrun/RUN.fdf -e -1.0 -n 3 {folder}')

In the output *ECrun* folder you will find many files, among them the scattering states in the XSF format, ready for visualization with xcrysden:
```
sudo apt install xcrysden # Ubuntu/Linux install 
xcrysden --xsf ECrun/device.EC.1L_E-1.000.XSF
```
Select from the drop-down menu _Tools -> Data Grid -> OK_ and choose a reasonable isosurface value

**Questions:**
* What are the meanings of the `-e -1.0` and `-n 3` flags to *EigenChannels*?
* What are the symmetries of the three dominant scattering states?
* What scattering states are actually available for transport at the Fermi level in this model system?

## Calculation of vibrational modes, energies, and EPC ##

We are now ready to look at the vibrational modes. Again, `Inelastica` has a script (called `Phonons`) to analyze the data from the `FCrun*` and `OSrun` above to 
* construct the dynamical matrix **D**, 
* symmetrize and diagonalize **D** to obtain normal coordinates and mode energies,
* collect the *.TSHS* files to compute the EPC corresponding to each normal coordinate

The calculation proceeds as follows:

In [ ]:
folder = 'PHrun'
#os.system(f'Phonons -c --FCfirst=7 --FClast=8 -F 5 -L 10 --FCwildcard=FCrun {folder}')

**Questions:**
* Look at the log file from the run. What has been computed?
* What changes if one excludes the `-c` flag from the call?
* What are the meanings of the other flags above?
* Try to do another calculation (say directed to output directory `PHrun_2`) where data from the two parallelized directories `FCrun_*` are used instead.

The *Phonons* script writes several files, among them basically all quantities to a file in the netCDF4 file format. Let us use it to plot the vibrational energies:

In [ ]:
PH = nc4.Dataset(f'{folder}/<Output.nc>', 'r');
hw = PH.variables['hw'][:];
plt.plot(hw, 'o-');
plt.xlabel('Mode index');
plt.ylabel('Mode energy (eV)');

**Question**
* Why are there 6 modes?
* What is the meaning of negative mode energies?

To understand better the character of each mode one can use `xcrysden` to visualize the normal coordinate vectors, e.g., 
```
xcrysden --xsf PHrun/Output.mol.axsf
```
Change from the menu _Modify -> Force Settings -> Length Factor = 2_. Enable from the menu _Display -> Forces_.

We can also develop an understanding of the different modes without a 3D visualizer, by reading the normal coordinate matrix of displacement amplitudes. The format is (mode index, atom index): 

In [ ]:
mode = 0
dx, dy, dz = PH.variables['U'][mode].T
plt.plot(dx, 'o', label='dx');
plt.plot(dy, 'o', label='dy');
plt.plot(dz, 'o', label='dz');
plt.legend()
plt.title(f'Normal mode = {mode}');
plt.xlabel('Atom index');
plt.ylabel('Displacement amplitude');

**Questions**
* Which modes are longitudinal with respect to the system axis?
* Which ones are transversal?
* Which pairs of modes are degenerate? And why?
* The conventional naming of the modes are CO stretch, CO center-of-mass motion, and frustrated/hindered translation and rotations. Can you establish the links?

## Calculation of IETS spectrum ##
With the complete `transiesta` setup and the computed EPCs we can proceed to compute the inelastic transport characteristics with `Inelastica` (both the package name and the name of the script that computes I-V curves etc).

In [ ]:
folder = 'INrun'
#os.system(f'Inelastica -p PHrun/Output.nc -f TSrun/RUN.fdf -F 5 -L 10 {folder}')

The main data file from `Inelastica` is written to a single `netCDF` file, from which we can plot various results:

In [ ]:
IN = nc4.Dataset(f'{folder}/<Output.nc>', 'r')
V = IN.variables['V'][:]
I = IN.variables['I'][:]
dI = IN.variables['dI'][:] # first derivative wrt V
ddI = IN.variables['ddI'][:] # second derivative wrt V
plt.plot(V, I, label=r'$I(V)$');
plt.plot(V, dI, label=r'$dI/dV$');
plt.plot(V, ddI, label=r'$d^2I/dV^2$');
plt.legend()
plt.xlabel('Bias voltage (V)');

Since the inelastic signals are often very small, it is conventional to plot the so-called IETS signal, defined as the second derivative normalized to the first, i.e., `IETS = (ddI/dI)`:

In [ ]:
IN = nc4.Dataset(f'{folder}/<Output.nc>', 'r')
IETS = IN.variables['IETS'][:]
plt.plot(V, IETS);
plt.plot(hw, 0 * hw, 'o');
plt.xlabel('Bias voltage (V)');
plt.ylabel('IETS (1/V)');

**Questions**
* Which modes are predicted to be vibrationally active in this setup?

## IETS signals from transverse modes ## 
In reality, STM-IETS on CO on a Cu surface with a metal tip shows that the frustrated translation (FT) and frustrated rotation (FR) modes are the most IETS active. This is due to the frontier orbitals that are dominated by the molecular \pi-states (rotationally odd symmmetry with respect to the transport axis). However, in our simple model we only have the \sigma-type state available (rotationally symmetric) at the Fermi energy.

A simple, qualitative "hack" to approach a more realistic description of IETS of CO, we can shift the Fermi down into the _d_-band region. The position of the Fermi level can be shifted by hand in the `Inelastica` call. Let us try this as follows:

In [ ]:
folder = 'INrun_shift'
energy = -1.0
Vmax = 0.050
Vrms = 0.001
#os.system(f'Inelastica -p PHrun/<Output.nc> -f TSrun/RUN.fdf -F 5 -L 10 -e {energy} -V {Vrms} -v {Vmax} {folder}')

In [ ]:
IN = nc4.Dataset(f'{folder}/<Output.nc>', 'r')
V = IN.variables['V'][:]
IETS = IN.variables['IETS'][:]
plt.plot(V, IETS);
plt.xlabel('Bias voltage (V)');
plt.ylabel('IETS (1/V)');

**Questions:**
* What is the meaning of `-V {Vrms}`? What happens if you change its value?
* How does the electronic temperature affects the IETS?
* Try to add the flag `-H` to the `Inelastica` call above. What does it do?

# Further directions and possibilities #

## Isotope shifts ##

The vibrational modes of CO depends on the masses. By default, Inelastica assumes the atoms are the isotopes 12C and 16O. However, it is possible with Inelastica to specify different masses for the atoms: Here is an example where the masses of atom indices 6 and 7 are changed for 13C and 18O:

In [ ]:
folder = 'PHrun_isotope'
C13 = [6, 13.0]
O18 = [7, 18.0]
#os.system(f'Phonons -c --FCfirst=7 --FClast=8 -F 5 -L 10 --Isotopes "[{C13}, {O18}]" {folder}')

Next, compute IETS with `Inelastica` for these new EPCs via the flag `-p <path-to-isotope-Output.nc>`.

## Geometric effects in IETS ##

Play around with different device geometries and see how the IETS of a CO changes. These are some possible ideas:
* Move the tip away from the symmetry axis. The symmetry lowering removes constraints for possible vibrational transitions. Is this reflected in the IETS?
* The `-H` flag enables vibrational heating, which in turn depends on the magnitude of the (elastic) current. Try to bring the electrodes closer to each other to enhance the heating.

## Bloch's theorem and k-sampling of IETS ##

* Construct a setup with 2D or 3D Cu electrodes. Now, in principle a sampling of the (transverse) Brillouin zone is required (just for the electronic part, assuming we can still restrict CO vibrations to the Gamma point, ie. decoupled molecues). This can be achieved through (1) carrying out a loop over a `k`-points to compute EPCs and associated k-resolved IETS, before (2) performing the BZ average (see Inelastica script `kaverage-IETS`).  